In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

# Variables symboliques
y, y_hat, delta = sp.symbols('y y_hat delta')

# 1. Erreur Quadratique Moyenne (MSE)
mse = (y - y_hat)**2
grad_mse = sp.diff(mse, y_hat)
print("Gradient MSE :", grad_mse)

# 2. Entropie croisée binaire
bce = -(y*sp.log(y_hat) + (1 - y)*sp.log(1 - y_hat))
grad_bce = sp.diff(bce, y_hat)
print("Gradient BCE :", grad_bce)

# 3. Entropie croisée catégorielle
y_c, y_hat_c = sp.symbols('y_c y_hat_c')
cce = -y_c * sp.log(y_hat_c)
grad_cce = sp.diff(cce, y_hat_c)
print("Gradient CCE (pour une classe) :", grad_cce)

# 4. Perte de Huber
huber = sp.Piecewise(
    ((1/2)*(y - y_hat)**2, sp.Abs(y - y_hat) <= delta),
    (delta * (sp.Abs(y - y_hat) - delta/2), sp.Abs(y - y_hat) > delta)
)
grad_huber = sp.diff(huber, y_hat)
print("Gradient Huber :", grad_huber)

# Sigmoïde stable pour éviter overflow/underflow
def stable_sigmoid(x):
    pos = x >= 0
    neg = ~pos
    result = np.empty_like(x, dtype=np.float64)
    result[pos] = 1 / (1 + np.exp(-x[pos]))
    exp_x = np.exp(x[neg])
    result[neg] = exp_x / (1 + exp_x)
    return result

def mse_np(e):
    return e**2

def bce_np(y, y_hat):
    y_hat = np.clip(y_hat, 1e-10, 1 - 1e-10)  # éviter log(0)
    return -(y*np.log(y_hat) + (1 - y)*np.log(1 - y_hat))

def cce_np(y_c, y_hat_c):
    y_hat_c = np.clip(y_hat_c, 1e-10, 1 - 1e-10)
    return -y_c * np.log(y_hat_c)

def huber_np(e, delta=1.0):
    return np.where(np.abs(e) <= delta, 0.5 * e**2, delta * (np.abs(e) - delta/2))

e = np.linspace(-10, 10, 600)

plt.figure(figsize=(12, 8))
plt.plot(e, mse_np(e), label='Erreur Quadratique Moyenne')

y_hat_bce = stable_sigmoid(e)
plt.plot(e, bce_np(1, y_hat_bce), label='Entropie croisée binaire (y=1)')

y_hat_cce = stable_sigmoid(e)
plt.plot(e, cce_np(1, y_hat_cce), label='Entropie croisée catégorielle (classe 1)')

plt.plot(e, huber_np(e, delta=1.0), label='Perte de Huber')

plt.title("Comparaison des fonctions de perte")
plt.xlabel("Erreur (y - ŷ)")
plt.ylabel("Valeur de la perte")
plt.ylim(0, 10)  # Limitation pour une meilleure visibilité
plt.legend()
plt.grid(True)
plt.show()
